In [3]:
sc

In [4]:
from pyspark.sql import SQLContext
sqlc = SQLContext(sc)

## Spark ML Tuning

### Elements:
- Model (Estimator or Pipeline)
- Set of ParamMaps (to perform the grid search) - you should use the ***ParamGridBuilder*** utility
- Evaluator (to assess the fitness of the model)

## Cross-Validation

- Splits the dataset into K folds
- Each fold is splitted into a training (2/3) and a test (1/3) sets
- It will fit K models and compute the average of the K evaluation metrics (according to the Evaluator)
- Based on the metrics, it will determine the best set of parameters
- Then it will fit the model one final time, using this set of parameters and the whole dataset
- This is a VERY computationally expensive 

In [7]:
!rm -rf metastore_db/*.lck

training = sqlc.createDataFrame([
    (0, "a b c d e spark", 1.0),
    (1, "b d", 0.0),
    (2, "spark f g h", 1.0),
    (3, "hadoop mapreduce", 0.0),
    (4, "b spark who", 1.0),
    (5, "g d a y", 0.0),
    (6, "spark fly", 1.0),
    (7, "was mapreduce", 0.0),
    (8, "e spark program", 1.0),
    (9, "a e c l", 0.0),
    (10, "spark compile", 1.0),
    (11, "hadoop software", 0.0)
], ["id", "text", "label"])

In [8]:
from pyspark.ml import Pipeline
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.feature import HashingTF, Tokenizer

tokenizer = Tokenizer(inputCol="text", outputCol="words")

hashingTF = HashingTF(inputCol=tokenizer.getOutputCol(), outputCol="features")

lr = LogisticRegression(maxIter=10)

pipeline = Pipeline(stages=[tokenizer, hashingTF, lr])

In [9]:
from pyspark.ml.tuning import ParamGridBuilder

paramGrid = ParamGridBuilder() \
    .addGrid(hashingTF.numFeatures, [10, 100, 1000]) \
    .addGrid(lr.regParam, [0.1, 0.01]) \
    .build()

In [11]:
paramGrid

[{Param(parent=u'HashingTF_4a57a3957b38db7942a4', name='numFeatures', doc='number of features.'): 10,
  Param(parent=u'LogisticRegression_46a482905b563c13383e', name='regParam', doc='regularization parameter (>= 0).'): 0.1},
 {Param(parent=u'HashingTF_4a57a3957b38db7942a4', name='numFeatures', doc='number of features.'): 10,
  Param(parent=u'LogisticRegression_46a482905b563c13383e', name='regParam', doc='regularization parameter (>= 0).'): 0.01},
 {Param(parent=u'HashingTF_4a57a3957b38db7942a4', name='numFeatures', doc='number of features.'): 100,
  Param(parent=u'LogisticRegression_46a482905b563c13383e', name='regParam', doc='regularization parameter (>= 0).'): 0.1},
 {Param(parent=u'HashingTF_4a57a3957b38db7942a4', name='numFeatures', doc='number of features.'): 100,
  Param(parent=u'LogisticRegression_46a482905b563c13383e', name='regParam', doc='regularization parameter (>= 0).'): 0.01},
 {Param(parent=u'HashingTF_4a57a3957b38db7942a4', name='numFeatures', doc='number of features.')

In [13]:
from pyspark.ml.tuning import CrossValidator

crossval = CrossValidator(estimator=pipeline,
                          estimatorParamMaps=paramGrid,
                          evaluator=BinaryClassificationEvaluator(),
                          numFolds=3)

cvModel = crossval.fit(training)

In [16]:
cvModel.avgMetrics

[1.75, 1.75, 1.75, 1.75, 1.8333333333333335, 1.8333333333333335]

In [49]:
cvModel.bestModel

PipelineModel_457d82a119e332275598

In [20]:
cvModel.bestModel.stages

[Tokenizer_448c9b3d5cbe87b540d1,
 HashingTF_4a57a3957b38db7942a4,
 LogisticRegression_46a482905b563c13383e]

In [23]:
lr_best = cvModel.bestModel.stages[2]

In [24]:
lr_best.coefficients

SparseVector(1000, {6: -1.4383, 66: -0.7317, 94: -0.4712, 105: 2.0311, 170: -0.1318, 181: -0.7851, 217: 0.9701, 234: -0.7317, 248: 0.755, 282: 0.755, 315: -0.7966, 361: 0.2486, 417: -0.0229, 463: 1.0866, 644: 0.8376, 695: 1.0866, 722: 0.2602, 878: 0.534, 953: -0.7851})

In [26]:
lr_summary = lr_best.summary

In [27]:
lr_summary.areaUnderROC

1.0

In [29]:
test = sqlc.createDataFrame([
    (4L, "spark i j k"),
    (5L, "l m n"),
    (6L, "mapreduce spark"),
    (7L, "apache hadoop")
], ["id", "text"])

In [30]:
prediction = cvModel.transform(test)

selected = prediction.select("id", "text", "probability", "prediction")

for row in selected.collect():
    print(row)

Row(id=4, text=u'spark i j k', probability=DenseVector([0.2661, 0.7339]), prediction=1.0)
Row(id=5, text=u'l m n', probability=DenseVector([0.9209, 0.0791]), prediction=0.0)
Row(id=6, text=u'mapreduce spark', probability=DenseVector([0.4429, 0.5571]), prediction=1.0)
Row(id=7, text=u'apache hadoop', probability=DenseVector([0.8584, 0.1416]), prediction=0.0)


## Train-Validation Split

- It uses the entire dataset
- The dataset is splitted into a training and a test sets according to the ***trainRatio*** parameter
- It will fit a model for each set of parameters and evaluate its metrics (according to the Evaluator)
- Based on the metrics, it will determine the best set of parameters
- Then it will fit the model one final time, using this set of parameters and the whole dataset
- This is a much less expensive, but it may not yield good results if the dataset is not large enough

In [51]:
from pyspark.ml.regression import LinearRegression

data = sqlc.read.format("libsvm").load("/home/ubuntu/spark/data/mllib/sample_linear_regression_data.txt")

train, test = data.randomSplit([0.7, 0.3])

lr = LinearRegression(maxIter=20, regParam=0.1)

In [52]:
from pyspark.ml.tuning import ParamGridBuilder

paramGrid = ParamGridBuilder()\
    .addGrid(lr.regParam, [0.1, 0.01]) \
    .addGrid(lr.elasticNetParam, [0.0, 0.5, 1.0])\
    .build()

In [53]:
paramGrid

[{Param(parent=u'LinearRegression_40c59fea604826a0431d', name='elasticNetParam', doc='the ElasticNet mixing parameter, in range [0, 1]. For alpha = 0, the penalty is an L2 penalty. For alpha = 1, it is an L1 penalty.'): 0.0,
  Param(parent=u'LinearRegression_40c59fea604826a0431d', name='regParam', doc='regularization parameter (>= 0).'): 0.1},
 {Param(parent=u'LinearRegression_40c59fea604826a0431d', name='elasticNetParam', doc='the ElasticNet mixing parameter, in range [0, 1]. For alpha = 0, the penalty is an L2 penalty. For alpha = 1, it is an L1 penalty.'): 0.5,
  Param(parent=u'LinearRegression_40c59fea604826a0431d', name='regParam', doc='regularization parameter (>= 0).'): 0.1},
 {Param(parent=u'LinearRegression_40c59fea604826a0431d', name='elasticNetParam', doc='the ElasticNet mixing parameter, in range [0, 1]. For alpha = 0, the penalty is an L2 penalty. For alpha = 1, it is an L1 penalty.'): 1.0,
  Param(parent=u'LinearRegression_40c59fea604826a0431d', name='regParam', doc='regu

In [54]:
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.tuning import TrainValidationSplit

tvs = TrainValidationSplit(estimator=lr,
                           estimatorParamMaps=paramGrid,
                           evaluator=RegressionEvaluator(),
                           trainRatio=0.8)

model = tvs.fit(train)

In [55]:
model.bestModel

LinearRegression_40c59fea604826a0431d

In [56]:
model.bestModel.coefficients

DenseVector([-0.1492, 0.2969, -0.6552, 1.5598, 0.1965, 1.3713, 0.0, -0.2379, -0.7382, 0.4184])

In [57]:
model.bestModel.summary.r2

0.021874120623119664

In [73]:
prediction = model.transform(test)

In [72]:
prediction.toPandas()[:5]

,label,features,prediction
0,-28.046018,"(0.949330819585, 0.328521466154, 0.74930027801...",-1.569149
1,-23.487440,"(-0.519535443126, 0.808035794841, 0.8498613208...",-0.796819
2,-20.057483,"(-0.320505782811, 0.51605972927, 0.45215640988...",0.144287
3,-19.872991,"(-0.932581077292, -0.641147114733, 0.994921629...",0.016299
4,-19.402336,"(0.462288625222, -0.902975525943, 0.7442695642...",-0.602021
